In [2]:
#r "nuget: Apache.Ignite, 2.15.0"
#r "nuget: Apache.Ignite.Linq, 2.15.0"

Installed Packages Apache.Ignite, 2.15.0 Apache.Ignite.Linq, 2.15.0

In [3]:
using System;
using System.Collections.Generic;
using System.Net.Http;
using System.Threading.Tasks;
using Apache.Ignite.Core;
using Apache.Ignite.Core.Cache;
using Apache.Ignite.Core.Cache.Configuration;

In [4]:
class Country
{
    public int Id { get; set; }
    public string Name { get; set; }
    public string Iso3 { get; set; }
    public string Iso2 { get; set; }
    public string NumericCode { get; set; }
    public string PhoneCode { get; set; }
    public string Capital { get; set; }
    public string Currency { get; set; }
    public string CurrencyName { get; set; }
    public string CurrencySymbol { get; set; }
    public string Tld { get; set; }
    public string Native { get; set; }
    public string Region { get; set; }
    public string RegionId { get; set; }
    public string Subregion { get; set; }
    public string SubregionId { get; set; }
    public string Nationality { get; set; }
    public List<TimeZone> Timezones { get; set; }
    public Translations Translations { get; set; }
    public string Latitude { get; set; }
    public string Longitude { get; set; }
    public string Emoji { get; set; }
    public string EmojiU { get; set; }
}

class TimeZone
{
    public string ZoneName { get; set; }
    public int GmtOffset { get; set; }
    public string GmtOffsetName { get; set; }
    public string Abbreviation { get; set; }
    public string TzName { get; set; }
}

class Translations
{
    public string Kr { get; set; }
    public string PtBr { get; set; }
    public string Pt { get; set; }
    public string Nl { get; set; }
    public string Hr { get; set; }
    public string Fa { get; set; }
    public string De { get; set; }
    public string Es { get; set; }
    public string Fr { get; set; }
    public string Ja { get; set; }
    public string It { get; set; }
    public string Cn { get; set; }
    public string Tr { get; set; }
}

In [ ]:
async Task<string> FetchJsonDataAsync(string url)
{
    using (var httpClient = new HttpClient())
    {
        return await httpClient.GetStringAsync(url);
    }
}

var countriesJsonUrl = "https://raw.githubusercontent.com/dr5hn/countries-states-cities-database/master/countries.json";
var countriesJson = await FetchJsonDataAsync(countriesJsonUrl);

In [ ]:
var igniteConfig = new IgniteConfiguration
{
    CacheConfiguration = new[]
    {
        new CacheConfiguration("countriesCache")
        {
            QueryEntities = new[]
            {
                new QueryEntity
                {
                    KeyType = typeof(int),
                    ValueType = typeof(Country),
                    Fields = new[]
                    {
                        new QueryField("Id", typeof(int)),
                        new QueryField("Name", typeof(string)),
                        new QueryField("Region", typeof(string)),
                        new QueryField("Subregion", typeof(string)),
                        new QueryField("Capital", typeof(string)),
                        new QueryField("Currency", typeof(string)),
                        // Add other fields for indexing
                    }
                }
            },
            SqlSchema = "PUBLIC", // Change the schema if needed
            Backups = 1, // Number of backups for data redundancy
            WriteSynchronizationMode = CacheWriteSynchronizationMode.FullSync,
            AtomicityMode = CacheAtomicityMode.Transactional,
            CacheMode = CacheMode.Partitioned, // Change to CacheMode.Replicated if needed
            OnheapCacheEnabled = true,
            QueryParallelism = 2, // Adjust based on the available CPU cores
            // Add more cache configuration options as needed
        }
    },
    // Add more Ignite configuration options as needed
};


In [ ]:

using (var ignite = Ignition.Start(igniteConfig))
{
    // Create a cache
    var cache = ignite.GetOrCreateCache<int, Country>("countriesCache");

    // Deserialize JSON data and insert into the cache
    var countries = Newtonsoft.Json.JsonConvert.DeserializeObject<List<Country>>(countriesJson);

    foreach (var country in countries)
    {
        cache.Put(country.Id, country);
    }

    // Display the number of countries in the cache
    Console.WriteLine($"Number of countries in cache: {cache.GetSize()}");

    // Sample query
    var query = cache.Query(new Apache.Ignite.Core.Cache.Query.SqlFieldsQuery("SELECT Name FROM Country WHERE Region = 'Asia'"));

    foreach (var result in query)
    {
        Console.WriteLine($"Country in Asia: {result[0]}");
    }

    // Sample transaction
    using (var tx = ignite.GetTransactions().TxStart())
    {
        // Perform transactional operations
        var countryToUpdate = cache.Get(1);
        countryToUpdate.Name = "Updated Country";
        cache.Put(1, countryToUpdate);
        Console.WriteLine("Sample transaction");
        // Commit the transaction
        tx.Commit();
    }

    // Sample concurrency control (optimistic)
    var updatedCountry = new Country { Id = 1, Name = "Updated Value" };

    // Use GetAndPutIfAbsent as a workaround for optimistic concurrency control
    var existingCountry = cache.GetAndPutIfAbsent(1, updatedCountry);

    if (existingCountry == null)
    {
        Console.WriteLine("Successfully updated value");
    }
    else
    {
        Console.WriteLine("Failed to update due to concurrency control");
    }
}



[07:29:41,455][WARNING][Thread-2][G] Ignite work directory is not provided, automatically resolved to: /Users/kiettran/Projects/weather-db/analysis/ignite/work
[07:29:41,457][INFO][Thread-2][IgniteKernal] 

>>>    __________  ________________  
>>>   /  _/ ___/ |/ /  _/_  __/ __/  
>>>  _/ // (7 7    // /  / / / _/    
>>> /___/\___/_/|_/___/ /_/ /___/   
>>> 
>>> ver. 2.15.0#20230425-sha1:f98f7f35
>>> 2023 Copyright(C) Apache Software Foundation
>>> 
>>> Ignite documentation: https://ignite.apache.org

[07:29:41,457][INFO][Thread-2][IgniteKernal] Config URL: n/a
[07:29:41,458][INFO][Thread-2][IgniteKernal] IgniteConfiguration [igniteInstanceName=null, pubPoolSize=8, svcPoolSize=8, callbackPoolSize=8, stripedPoolSize=8, sysPoolSize=8, mgmtPoolSize=4, dataStreamerPoolSize=8, utilityCachePoolSize=8, utilityCacheKeepAliveTime=60000, p2pPoolSize=2, qryPoolSize=8, buildIdxPoolSize=2, igniteHome=null, igniteWorkDir=/Users/kiettran/Projects/weather-db/analysis/ignite/work, mbeanSrv=com.sun.jm

Number of countries in cache: 250
Country in Asia: Armenia
Country in Asia: Azerbaijan
Country in Asia: Bahrain
Country in Asia: Bhutan
Country in Asia: Hong Kong S.A.R.
Country in Asia: Indonesia
Country in Asia: Iraq
Country in Asia: Israel
Country in Asia: Kazakhstan
Country in Asia: South Korea
Country in Asia: Kyrgyzstan
Country in Asia: Macau S.A.R.
Country in Asia: Malaysia
Country in Asia: Mongolia
Country in Asia: Nepal
Country in Asia: Oman
Country in Asia: Philippines
Country in Asia: Saudi Arabia
Country in Asia: Sri Lanka
Country in Asia: Taiwan
Country in Asia: Turkmenistan
Country in Asia: Uzbekistan
Country in Asia: Vietnam
Country in Asia: Afghanistan
Country in Asia: Bangladesh
Country in Asia: Brunei
Country in Asia: Cambodia
Country in Asia: China
Country in Asia: East Timor
Country in Asia: Georgia
Country in Asia: India
Country in Asia: Iran
Country in Asia: Japan
Country in Asia: Jordan
Country in Asia: North Korea
Country in Asia: Kuwait
Country in Asia: Laos
Co

[07:29:42,923][INFO][Thread-2][GridTcpRestProtocol] Command protocol successfully stopped: TCP binary
[07:29:42,928][INFO][Thread-2][GridCacheProcessor] Stopped cache [cacheName=countriesCache]
[07:29:42,929][INFO][Thread-2][GridCacheProcessor] Stopped cache [cacheName=ignite-sys-cache]
[07:29:42,943][INFO][Thread-2][IgniteKernal] 

>>> +----------------------------------------------------------------------------------+
>>> Ignite ver. 2.15.0#20230425-sha1:f98f7f35de6dc76a9b69299154afaa2139a5ec6d stopped OK
>>> +----------------------------------------------------------------------------------+
>>> Grid uptime: 00:00:00.116




# SQLite

In [1]:
#r "nuget: System.Data.SQLite, 1.0.118"

Installed Packages System.Data.SQLite, 1.0.118

In [ ]:

// Import necessary libraries
using System;
using System.Collections.Generic;
using System.Data.SQLite;
using System.IO;
using Newtonsoft.Json.Linq;

// Create a SQLite database connection
string databasePath = "countries.db";
string connectionString = $"Data Source={databasePath};Version=3;";
using (SQLiteConnection connection = new SQLiteConnection(connectionString))
{
    // Create a table to store country data
    string createTableQuery = @"
        CREATE TABLE IF NOT EXISTS Countries (
            Id INTEGER PRIMARY KEY,
            Name TEXT,
            Iso3 TEXT,
            Iso2 TEXT,
            NumericCode TEXT,
            PhoneCode TEXT,
            Capital TEXT,
            Currency TEXT,
            CurrencyName TEXT,
            CurrencySymbol TEXT,
            Tld TEXT,
            Native TEXT,
            Region TEXT,
            RegionId TEXT,
            Subregion TEXT,
            SubregionId TEXT,
            Nationality TEXT,
            Timezones TEXT,
            Translations TEXT,
            Latitude TEXT,
            Longitude TEXT,
            Emoji TEXT,
            EmojiU TEXT
        );
    ";
    connection.Open();
    using (SQLiteCommand createTableCommand = new SQLiteCommand(createTableQuery, connection))
    {
        createTableCommand.ExecuteNonQuery();
    }

    // Indexing structures
    string createIndexQuery = "CREATE INDEX IF NOT EXISTS idx_region ON Countries(Region);";
    using (SQLiteCommand createIndexCommand = new SQLiteCommand(createIndexQuery, connection))
    {
        createIndexCommand.ExecuteNonQuery();
    }

    // Read JSON data from the provided URL using HttpClient
    string jsonDataUrl = "https://raw.githubusercontent.com/dr5hn/countries-states-cities-database/master/countries.json";
    string jsonData;
    using (HttpClient httpClient = new HttpClient())
    {
        jsonData = await httpClient.GetStringAsync(jsonDataUrl);
    }

    // Deserialize JSON data into a list of Country objects
    List<Country> countries = Newtonsoft.Json.JsonConvert.DeserializeObject<List<Country>>(jsonData);

    // Insert country data into the SQLite database
    string insertDataQuery = "INSERT INTO Countries (Id, Name, Iso3, Iso2, NumericCode, PhoneCode, Capital, Currency, CurrencyName, CurrencySymbol, Tld, Native, Region, RegionId, Subregion, SubregionId, Nationality, Timezones, Translations, Latitude, Longitude, Emoji, EmojiU) VALUES (@Id, @Name, @Iso3, @Iso2, @NumericCode, @PhoneCode, @Capital, @Currency, @CurrencyName, @CurrencySymbol, @Tld, @Native, @Region, @RegionId, @Subregion, @SubregionId, @Nationality, @Timezones, @Translations, @Latitude, @Longitude, @Emoji, @EmojiU);";
    using (SQLiteTransaction transaction = connection.BeginTransaction())
    {
        foreach (Country country in countries)
        {
            using (SQLiteCommand insertCommand = new SQLiteCommand(insertDataQuery, connection, transaction))
            {
                insertCommand.Parameters.AddWithValue("@Id", country.Id);
                insertCommand.Parameters.AddWithValue("@Name", country.Name);
                insertCommand.Parameters.AddWithValue("@Iso3", country.Iso3);
                insertCommand.Parameters.AddWithValue("@Iso2", country.Iso2);
                insertCommand.Parameters.AddWithValue("@NumericCode", country.NumericCode);
                insertCommand.Parameters.AddWithValue("@PhoneCode", country.PhoneCode);
                insertCommand.Parameters.AddWithValue("@Capital", country.Capital);
                insertCommand.Parameters.AddWithValue("@Currency", country.Currency);
                insertCommand.Parameters.AddWithValue("@CurrencyName", country.CurrencyName);
                insertCommand.Parameters.AddWithValue("@CurrencySymbol", country.CurrencySymbol);
                insertCommand.Parameters.AddWithValue("@Tld", country.Tld);
                insertCommand.Parameters.AddWithValue("@Native", country.Native);
                insertCommand.Parameters.AddWithValue("@Region", country.Region);
                insertCommand.Parameters.AddWithValue("@RegionId", country.RegionId);
                insertCommand.Parameters.AddWithValue("@Subregion", country.Subregion);
                insertCommand.Parameters.AddWithValue("@SubregionId", country.SubregionId);
                insertCommand.Parameters.AddWithValue("@Nationality", country.Nationality);
                insertCommand.Parameters.AddWithValue("@Timezones", Newtonsoft.Json.JsonConvert.SerializeObject(country.Timezones));
                insertCommand.Parameters.AddWithValue("@Translations", Newtonsoft.Json.JsonConvert.SerializeObject(country.Translations));
                insertCommand.Parameters.AddWithValue("@Latitude", country.Latitude);
                insertCommand.Parameters.AddWithValue("@Longitude", country.Longitude);
                insertCommand.Parameters.AddWithValue("@Emoji", country.Emoji);
                insertCommand.Parameters.AddWithValue("@EmojiU", country.EmojiU);

                insertCommand.ExecuteNonQuery();
            }
        }

        transaction.Commit();
    }

    // Query and print some data from the database
    string selectQuery = "SELECT * FROM Countries WHERE Region = 'Asia' LIMIT 5;";
    using (SQLiteCommand selectCommand = new SQLiteCommand(selectQuery, connection))
    {
        using (SQLiteDataReader reader = selectCommand.ExecuteReader())
        {
            Console.WriteLine("Query Results:");
            while (reader.Read())
            {
                Console.WriteLine($"Id: {reader["Id"]}, Name: {reader["Name"]}, Capital: {reader["Capital"]}, Region: {reader["Region"]}, Subregion: {reader["Subregion"]}");
            }
        }
    }
}


